# Project 3 Notebook

# BART Map

![Bart Map](bart_map.png)

# Included Modules and Packages

In [286]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

import json

import gmaps
import gmaps.geojson_geometries

from geographiclib.geodesic import Geodesic

# Supporting code

In [287]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [288]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [289]:
cursor = connection.cursor()

In [290]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [291]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [292]:
#
#  Given two points in (latitude, longitude) format, calculate the distance between them in miles
#

def my_calculate_distance(point_1, point_2):
    "Given two points in (latitude, longitude) format, calculate the distance between them in miles"
    
    geod = Geodesic.WGS84


    g = geod.Inverse(point_1[0], point_1[1], point_2[0], point_2[1])
    miles = g['s12'] / 1000 * 0.621371
    
    return miles

## Connect to Google Maps using your api key;  edit the file gmap_api_key.txt and put in your api key

In [293]:
f = open('gmap_api_key.txt', 'r') # You need this file in the same folder that contains this notebook
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

In [294]:
# Getting Berkeley's store information and location

rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT * 
FROM stores
WHERE city = 'Berkeley'

"""

store = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
store_coordinates = (store['latitude'][0], store['longitude'][0])

In [295]:
# Function to get zip codes with a minimum quantity of customers
# Within n miles from a station

def my_station_get_zips(station, miles, min_customers = 0):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
    distance_store = round(my_calculate_distance(store_coordinates, point),2)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    
    query = "SELECT cu.zip, z.population, count(cu.customer_id)"
    query += " FROM zip_codes AS z LEFT OUTER JOIN customers AS cu ON cu.zip = z.zip"
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " GROUP BY cu.zip, z.population"
    query += " HAVING count(cu.customer_id) >= " + str(min_customers)
    query += " order by count(cu.customer_id) DESC "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    print("\n-------------------------------------------------------------------------------")
    print("  Zip Codes within " + str(miles) + " mile(s) of " + station + " BART Station")
    print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    total_customers = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        customers = row[2]
        print("zip:", zip, "  population:", f'{population:10,}', "  customers:", f'{customers:5,}')
        total_population += population
        total_customers += customers
        
    
    print("\n-------------------------------------------------------------------------------")
    print("  Total Population: ", f'{total_population:10,}')
    print("  Total Customers: ", f'{total_customers:11,}')
    print("  Distance from Store: ", f'{distance_store:7,}')
    print("-------------------------------------------------------------------------------")

# Dropping tables if they exist

In [296]:
connection.rollback()

query = """

DROP TABLE IF EXISTS stations;
DROP TABLE IF EXISTS lines;
DROP TABLE IF EXISTS travel_times;

"""

cursor.execute(query)

connection.commit()


# Creating tables

In [297]:
connection.rollback()

query = """

CREATE TABLE stations (
  station VARCHAR(32),
  latitude NUMERIC(9,6),
  longitude NUMERIC(9,6),
  transfer_time NUMERIC(3),
  PRIMARY KEY (station)
);

CREATE TABLE lines (
  line VARCHAR(6),
  sequence NUMERIC(2),
  station VARCHAR(32),
  PRIMARY KEY (line, sequence)
);

CREATE TABLE travel_times (
  station_1 VARCHAR(32),
  station_2 VARCHAR(32),
  travel_time NUMERIC(3),
  PRIMARY KEY (station_1, station_2)
);


"""

cursor.execute(query)

connection.commit()

# Loading database tables

In [298]:
# Need to change this cell to the specific route where you have these files

connection.rollback()

query = """

COPY stations (station, latitude, longitude, transfer_time)
FROM '/user/projects/project-3-Alteras1/exercise/stations.csv' delimiter ',' NULL '' csv header;

COPY lines (line, sequence, station)
FROM '/user/projects/project-3-Alteras1/exercise/lines.csv' delimiter ',' NULL '' csv header;

COPY travel_times (station_1, station_2, travel_time)
FROM '/user/projects/project-3-Alteras1/exercise/travel_times.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

## Getting zip codes for each customer

In [299]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT z.latitude, z.longitude
FROM customers AS cu
     JOIN zip_codes AS z
         ON cu.zip = z.zip
         
WHERE cu.closest_store_id = 1
ORDER BY 1,2

"""

zip_cust = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
zip_cust

,latitude,longitude
0,37.4949,-122.2080
1,37.4949,-122.2080
2,37.4949,-122.2080
3,37.4949,-122.2080
4,37.4949,-122.2080
...,...,...
8133,38.1842,-122.2629
8134,38.1842,-122.2629
8135,38.1842,-122.2629
8136,38.1842,-122.2629


## Getting station information

In [300]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT * 
FROM stations
ORDER BY station;

"""

bart_stations = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

bart_markers = bart_stations[['latitude','longitude']]
station_names = bart_stations[['station']]
store_marker = store[['latitude','longitude']]

In [301]:
# Getting station and zip codes

for station in station_names['station']:
    my_station_get_zips(station, 1)


-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of 12th Street BART Station
-------------------------------------------------------------------------------

zip: 94612   population:     16,062   customers:   161

-------------------------------------------------------------------------------
  Total Population:      16,062
  Total Customers:          161
  Distance from Store:     3.63
-------------------------------------------------------------------------------

-------------------------------------------------------------------------------
  Zip Codes within 1 mile(s) of 16th Street Mission BART Station
-------------------------------------------------------------------------------

zip: 94103   population:     28,735   customers:    77
zip: 94114   population:     34,754   customers:    58

-------------------------------------------------------------------------------
  Total Population:      63,489
  Total Customers: 

# Visualizations

In [302]:
f = open('geojson_data/customer_zip_geojson.json')
customer_zip_geojson = json.load(f)
f.close()

fig = gmaps.figure(center=store_coordinates, map_type = 'HYBRID', zoom_level=10)

# Adding geometries for zip codes
geojson_layer = gmaps.geojson_layer(customer_zip_geojson, fill_color = 'white', fill_opacity = 0, 
                                    stroke_color = 'white', stroke_weight = 1, stroke_opacity = 0.7)
fig.add_layer(geojson_layer)

# Adding point markers for stations
station_marker_layer = gmaps.symbol_layer(bart_markers, hover_text = station_names['station'],
                                          fill_color = 'white', fill_opacity = 1, 
                                          stroke_color = 'blue', stroke_opacity = 1)
fig.add_layer(station_marker_layer)

# Adding customer heatmap by zipcode
heatmap_layer = gmaps.heatmap_layer(zip_cust)
heatmap_layer.point_radius = 25
fig.add_layer(heatmap_layer)

# Adding marker for the store location
fig.add_layer(gmaps.marker_layer(store_marker, info_box_content =["Berkeley ACME Store"]))

# Adding transit layer
fig.add_layer(gmaps.transit_layer())

fig

Figure(layout=FigureLayout(height='420px'))

In [303]:
f = open('geojson_data/customer_zip_geojson.json')
customer_zip_geojson = json.load(f)
f.close()

fig = gmaps.figure(center=store_coordinates, zoom_level=10)

# Adding geometries for zip codes
geojson_layer = gmaps.geojson_layer(customer_zip_geojson, fill_color = 'white', fill_opacity = 0, 
                                    stroke_color = 'gray', stroke_weight = 1, stroke_opacity = 0.7)
fig.add_layer(geojson_layer)

# Adding point markers for stations
station_marker_layer = gmaps.symbol_layer(bart_markers, hover_text = station_names['station'],
                                          fill_color = 'white', fill_opacity = 1, 
                                          stroke_color = 'blue', stroke_opacity = 1)
fig.add_layer(station_marker_layer)

# Adding customer heatmap by zipcode
heatmap_layer = gmaps.heatmap_layer(zip_cust)
heatmap_layer.point_radius = 25
fig.add_layer(heatmap_layer)

# Adding marker for the store location
fig.add_layer(gmaps.marker_layer(store_marker, info_box_content =["Berkeley ACME Store"]))

# Adding transit layer
fig.add_layer(gmaps.transit_layer())

fig

Figure(layout=FigureLayout(height='420px'))

# Neo4j

In [304]:
import neo4j

In [305]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","w205"))

In [306]:
session = driver.session(database="neo4j")

In [307]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [308]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [309]:
def neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")

In [310]:
def create_station_node_with_coords(station_name, latitude, longitude):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name,
                    location:point({latitude: $latitude, longitude: $longitude}),
                    line: []})
    
    """
    
    session.run(query, station_name=station_name, latitude=latitude, longitude=longitude)

In [311]:
def create_station_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)

In [312]:
def create_station_relationship_two_way(from_station, to_station, weight, line):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight, line: $line}]->(to),
           (to)-[:LINK {weight: $weight, line: $line}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight, line=line)

In [313]:
my_neo4j_wipe_out_database()

In [314]:
query = """

select * from stations order by station

"""

stations = my_select_query_pandas(query, True, True)

stations.apply(lambda s: create_station_node_with_coords(s['station'], s['latitude'], s['longitude']), axis=1)

neo4j_number_nodes_relationships()

-------------------------
  Nodes: 50
  Relationships: 0
-------------------------


In [315]:
def add_line_to_station(station, line):
    "adds a line to the line label for the station"
    
    query = """
    
    MATCH (s:Station)
    WHERE s.name = $station
    SET s.line = coalesce(s.line, []) + $line
    
    """
    
    session.run(query, station=station, line=line)

In [316]:
query = """

select * from lines order by station

"""

stations = my_select_query_pandas(query, True, True)

for station in stations.groupby('station')['line'].unique().to_dict().items():
    name = station[0]
    lines = station[1].tolist()
    
    add_line_to_station(name, lines)

In [317]:
query = """

SELECT 
    a.line,
    a.station as from_station,
    b.station as to_station,
    t.travel_time::integer
FROM
    lines a
    JOIN lines b
        ON a.line = b.line AND b.sequence = (a.sequence + 1)
    JOIN travel_times t
        ON (a.station = t.station_2 AND b.station = t.station_1)
            OR (a.station = t.station_1 AND b.station=t.station_2)
ORDER BY
    line,
    from_station,
    to_station

"""

lines_travel_times = my_select_query_pandas(query, True, True)

In [318]:
lines_travel_times

for index, row in lines_travel_times.iterrows():
    line = row[0]
    station_a = row[1]
    station_b = row[2]
    time = row[3]
    create_station_relationship_two_way(station_a, station_b, time, line)

In [319]:
def create_zipcode_node(zipcode, latitude, longitude, count):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Zip_Code {name: $zipcode,
                    location:point({latitude: $latitude, longitude: $longitude}),
                    count: $count})
    
    """
    
    session.run(query, zipcode=zipcode, latitude=latitude, longitude=longitude, count=count)

In [320]:
query = """

SELECT z.zip,
        z.latitude,
        z.longitude,
        count(cu.customer_id) 
FROM customers AS cu
     JOIN zip_codes AS z
         ON cu.zip = z.zip
         
WHERE cu.closest_store_id = 1
GROUP BY z.zip
ORDER BY count DESC

"""

zipcodes_customers = my_select_query_pandas(query, True, True)
zipcodes_customers.apply(lambda z: create_zipcode_node('ZC ' + z['zip'], z['latitude'], z['longitude'], z['count']), axis=1)


0      None
1      None
2      None
3      None
4      None
       ... 
139    None
140    None
141    None
142    None
143    None
Length: 144, dtype: object

In [321]:
query = """

MATCH (zip:Zip_Code)
MATCH (station:Station) WHERE point.distance(zip.location,station.location) < 4830
RETURN zip.name, station.name, point.distance(zip.location, station.location)/1000 AS distance_km, 1000/point.distance(zip.location, station.location) AS distance_inv
ORDER BY station.name

"""

my_neo4j_run_query_pandas(query)

,zip.name,station.name,distance_km,distance_inv
0,ZC 94608,12th Street,3.826299,0.261349
1,ZC 94607,12th Street,2.513580,0.397839
2,ZC 94606,12th Street,2.714935,0.368333
3,ZC 94501,12th Street,3.171903,0.315268
4,ZC 94610,12th Street,2.752890,0.363255
...,...,...,...,...
304,ZC 94606,West Oakland,4.641894,0.215429
305,ZC 94501,West Oakland,3.757666,0.266123
306,ZC 94610,West Oakland,4.699794,0.212775
307,ZC 94612,West Oakland,2.327587,0.429629


In [322]:
# connect zipcodes to all stations within 3 miles (~4830 meters)

query = """

MATCH (zip:Zip_Code)
MATCH (station:Station) WHERE point.distance(zip.location,station.location) < 4830
CREATE (zip)-[:Close_To {distance_km:point.distance(zip.location, station.location)/1000,
                        distance_inv:100000/point.distance(zip.location, station.location)
                        }]->(station)
CREATE (station)-[:Close_To {distance_km:point.distance(zip.location, station.location)/1000,
                        distance_inv:100000/point.distance(zip.location, station.location)
                        }]->(zip)

"""
session.run(query)

In [323]:
# connect all stations by closeness

query = """

MATCH (s:Station)
CALL {
    WITH s
    MATCH (s)-[:LINK]->(so:Station)
    RETURN collect(distinct so) AS other_stations
}
UNWIND other_stations as o
CREATE (s)-[:Close_To {distance_km: point.distance(s.location, o.location), distance_inv: 100000/point.distance(s.location, o.location)}]->(o)

"""
session.run(query)

In [324]:
# label all connections for easier graph projection

query = "MATCH (n)--(m) SET n:Connected"
session.run(query)

In [325]:
query = """CALL gds.graph.drop('distance_graph', false)"""

session.run(query)

query = """

CALL gds.graph.project(
    'distance_graph',
    'Connected',
    {Close_To: {orientation: 'UNDIRECTED'}},
    {relationshipProperties: 'distance_inv'}
)

"""

session.run(query)


In [326]:
query = """CALL gds.louvain.stats('distance_graph', {relationshipWeightProperty: 'distance_inv'})"""
my_neo4j_run_query_pandas(query)

,modularity,modularities,ranLevels,communityCount,communityDistribution,postProcessingMillis,preProcessingMillis,computeMillis,configuration
0,0.766764,"[0.6953195721364109, 0.76676375943006]",2,9,"{'p99': 27, 'min': 2, 'max': 27, 'mean': 15.0,...",2,0,96,"{'maxIterations': 10, 'seedProperty': None, 'c..."


In [ ]:
query = """CALL gds.louvain.write('distance_graph', {relationshipWeightProperty: 'distance_inv', writeProperty: 'communityId'})"""
session.run(query)

In [ ]:
query = """

MATCH (z:Zip_Code:Connected)
WITH z.communityId AS comId,
collect(distinct z.communityId) as communityIds
UNWIND communityIds as com
MATCH (s:Station:Connected) WHERE s.communityId = com
WITH s.communityId as sCom, com as com,
collect(s.name) AS stations
MATCH (p:Zip_Code:Connected) WHERE p.communityId = com
WITH p.communityId as pCom, stations as stations,
collect(p.name) AS zipcodes, sum(p.count) AS customers
RETURN pCom, stations, zipcodes, customers

"""
my_neo4j_run_query_pandas(query)